# Imports

In [1]:
import pandas as pd
from os import *
from os.path import isfile, join
from datetime import datetime
import numpy as np
import pandas as pd
import scipy as scipy
import seaborn as sns
import requests as rqs
import matplotlib.pyplot as plt
import random

plt.rcParams["figure.figsize"] = (16,9)

# Functions

In [2]:
# restituisce lista completa dei file nella cartella
def get_files_names(path):
    edifici = [f for f in listdir(path)][:-1]
    files = []
    
    for e in edifici:
        path2 = path + e + '\\'
        anni = [f for f in listdir(path2)]

        for a in anni:
            path3 = path2 + a + '\\'
            tmp = [path3+f for f in listdir(path3)]
            files += tmp
            
    return files

In [3]:
# fix delle estensioni sbagliate
def fix_files_extension():
    files = get_files_names('Dati Energia\\')
    for file in files:
        if file[-4:] == 'xltx':
            rename(file, file[:-4]+'xlsx')

#### Lettura file

In [4]:
# restituisce il df con tutti i dati
def open_full_df(files):
    dfs = []
    for file in files:
        dfs.append(open_df(file))
    df = pd.concat(dfs)
    return df


# apre un file e restituisce il df relativo
def open_df(file):
    if file[-3:] == 'csv':
        tmp = pd.read_csv(file, header = 0, names=cols, usecols=list(range(0,8)), index_col=False, sep=';', 
                    dtype={'consumo_attiva_prelevata': str, 'consumo_reattiva_induttiva_prelevata': str})
    
    else:
        tmp = pd.read_excel(file, header = 0, names=cols, usecols=list(range(0,8)), index_col=False, 
                    dtype={'consumo_attiva_prelevata': str, 'consumo_reattiva_induttiva_prelevata': str})
    
    for c in ['consumo_attiva_prelevata', 'consumo_reattiva_induttiva_prelevata']:
        tmp[c] = pd.to_numeric(tmp[c].str.replace(',', '.'))
    
    return tmp

#### Pulizia dataframe

In [5]:
# pulizia generale, richiama tutte le funzioni sotto
def clean_df(df, important_columns):
    df = df[df['pod'].notna()].copy()
    df = fix_datetime(df)
    df.rename(columns={'fl_ora_legale': 'ora_legale'}, inplace=True)
    df['pod'] = df['pod'].map({'IT012E00491869': 'u1', 'IT012E00491824': 'u6'})
    df = df[important_columns]
    df = drop_rows_duplicates(df)
    return df

In [6]:
# ricavo ora, quarto d'ora e dataora
def fix_datetime(df):
    df['ora'] = df['ora'].apply(lambda d: ''.join(['0' for i in range(6-len(str(int(d))))]) + str(int(d))  if d==d else np.nan)
    df['dataora'] = df.apply(lambda x: str(int(x['data']))+'-'+str(x['ora']), axis=1)
    df['dataora'] = pd.to_datetime(df['dataora'], format='%Y%m%d-%H%M%S')
    return df.copy()

# rimuove duplicati
def drop_rows_duplicates(df):
    df = df.drop_duplicates()
    return df.reset_index(drop=True)

#### Ricostruzione più comoda

In [7]:
def pivot_dataframe(df):
    tmp1 = df[df['pod']=='u1'][['dataora', 'ora_legale', 'consumo_attiva_prelevata', 
                                'consumo_reattiva_induttiva_prelevata', 'potenza_massima']].copy()
    tmp1 = tmp1.set_index('dataora')
    tmp1.rename(columns={'consumo_attiva_prelevata': 'consumo_attiva_u1', 
                         'consumo_reattiva_induttiva_prelevata': 'consumo_reattiva_u1',
                        'potenza_massima': 'potenza_massima_u1'}, inplace=True)
    tmp1 = tmp1[~tmp1.index.duplicated(keep='first')]


    tmp6 = df[df['pod']=='u6'][['dataora', 'consumo_attiva_prelevata', 
                                'consumo_reattiva_induttiva_prelevata', 'potenza_massima']].copy()
    tmp6 = tmp6.set_index('dataora')
    tmp6.rename(columns={'consumo_attiva_prelevata': 'consumo_attiva_u6', 
                         'consumo_reattiva_induttiva_prelevata': 'consumo_reattiva_u6',
                        'potenza_massima': 'potenza_massima_u6'}, inplace=True)
    tmp6 = tmp6[~tmp6.index.duplicated(keep='first')]


    return tmp1.join(tmp6)

#### Risoluzione valori mancanti

In [8]:
def fix_missing_values(df):
    # fix dei dati mancanti (interpolazione lineare per NaN puntuali; creazione dati fittizi per giugno 2020 u6)
    df = df.interpolate(method='linear')

    df18 = df[df.index.year==2018].copy()
    df19 = df[df.index.year==2019].copy()
    df20 = df[df.index.year==2020].copy()

#     a = np.std(df18['consumo_attiva_u6'])**2
#     b = np.std(df19['consumo_attiva_u6'])**2
#     c = np.std(df20['consumo_attiva_u6'])**2
#     m = c / ((a+b)/2)

#     a = df20[df20.index.month==5].tail(1)
#     b = df20[df20.index.month==7].head(1)
#     q = b['consumo_attiva_u6'][0]-a['consumo_attiva_u6'][0]
    
    m = 0.2
    q = 65

    df20_before = df20[df20.index.month<6].copy()
    df20_june = df20[df20.index.month==6].copy()
    df20_after = df20[df20.index.month>6].copy()

    j18 = list(df18['2018-06-04': '2018-07-03']['consumo_attiva_u6'])
    j19 = list(df19['2019-06-03': '2019-07-02']['consumo_attiva_u6'])
    j_avg = [(j18[i] + j19[i]) / 2 for i in range(len(j18))]

    l = [m*x+q for x in j_avg]
    j = 0.90
    for i in range(len(df20_june)):
        if random.random() < 0.2:
            l[i] += 5 * (random.random() - 0.5)
        l[i] *= j
        j += 0.2 / len(df20_june)
    
    df20_june['consumo_attiva_u6'] = l

    interpolated = pd.concat([df18, df19, df20_before, df20_june, df20_after])
    
    return interpolated

# Global variables

In [9]:
files = get_files_names('Dati Energia\\')

cols = ['pod', 'data', 'ora', 'fl_ora_legale', 'consumo_attiva_prelevata', 
        'consumo_reattiva_induttiva_prelevata', 'potenza_massima', 'tipo_dato']

important_columns = ['pod', 'dataora', 'ora_legale', 'consumo_attiva_prelevata',
                      'consumo_reattiva_induttiva_prelevata','potenza_massima']

# Code

In [10]:
### FIX MANUALI 
# colonne in più inutili nei file: u6giugno2020, u6luglio2020, ... non mi ricordo quali (sono 4 o 5)
# eliminazione di copia gennaio 2020 u1
# eliminazione righe 25 ottobre 2020 di troppo (entrambi edifici)

In [11]:
raw_df = open_full_df(files)
raw_df

,pod,data,ora,fl_ora_legale,consumo_attiva_prelevata,consumo_reattiva_induttiva_prelevata,potenza_massima,tipo_dato
0,IT012E00491869,20180801.0,0.0,2.0,102.00,43.80,547.2,E
1,IT012E00491869,20180801.0,1500.0,2.0,100.80,42.00,547.2,E
2,IT012E00491869,20180801.0,3000.0,2.0,103.20,43.80,547.2,E
3,IT012E00491869,20180801.0,4500.0,2.0,100.80,43.20,547.2,E
4,IT012E00491869,20180801.0,10000.0,2.0,98.40,42.60,547.2,E
...,...,...,...,...,...,...,...,...
2875,IT012E00491824,20200930.0,224500.0,2.0,87.45,30.25,NaN,E
2876,IT012E00491824,20200930.0,230000.0,2.0,87.45,28.60,NaN,E
2877,IT012E00491824,20200930.0,231500.0,2.0,86.90,28.60,NaN,E
2878,IT012E00491824,20200930.0,233000.0,2.0,86.90,28.60,NaN,E


In [12]:
df = clean_df(raw_df, important_columns)
df

,pod,dataora,ora_legale,consumo_attiva_prelevata,consumo_reattiva_induttiva_prelevata,potenza_massima
0,u1,2018-08-01 00:00:00,2.0,102.00,43.80,547.2
1,u1,2018-08-01 00:15:00,2.0,100.80,42.00,547.2
2,u1,2018-08-01 00:30:00,2.0,103.20,43.80,547.2
3,u1,2018-08-01 00:45:00,2.0,100.80,43.20,547.2
4,u1,2018-08-01 01:00:00,2.0,98.40,42.60,547.2
...,...,...,...,...,...,...
207547,u6,2020-09-30 22:45:00,2.0,87.45,30.25,NaN
207548,u6,2020-09-30 23:00:00,2.0,87.45,28.60,NaN
207549,u6,2020-09-30 23:15:00,2.0,86.90,28.60,NaN
207550,u6,2020-09-30 23:30:00,2.0,86.90,28.60,NaN


In [13]:
df = pivot_dataframe(df)
df.sort_index(inplace=True)
df

,ora_legale,consumo_attiva_u1,consumo_reattiva_u1,potenza_massima_u1,consumo_attiva_u6,consumo_reattiva_u6,potenza_massima_u6
dataora,,,,,,,
2018-01-01 00:00:00,1.0,64.8,21.6,446.4,93.50,38.50,888.8
2018-01-01 00:15:00,1.0,66.0,25.2,446.4,91.30,35.75,888.8
2018-01-01 00:30:00,1.0,62.4,22.2,446.4,91.85,36.30,888.8
2018-01-01 00:45:00,1.0,64.8,22.8,446.4,90.20,35.20,888.8
2018-01-01 01:00:00,1.0,66.0,24.0,446.4,91.85,37.40,888.8
...,...,...,...,...,...,...,...
2020-12-31 22:45:00,1.0,54.0,27.0,NaN,133.65,57.75,NaN
2020-12-31 23:00:00,1.0,53.4,27.0,NaN,133.65,57.75,NaN
2020-12-31 23:15:00,1.0,54.6,28.2,NaN,134.20,58.30,NaN


In [14]:
# df.to_csv('energia_cleaned.csv')

In [15]:
# resample orario
df = df.resample('H').mean()
df

,ora_legale,consumo_attiva_u1,consumo_reattiva_u1,potenza_massima_u1,consumo_attiva_u6,consumo_reattiva_u6,potenza_massima_u6
dataora,,,,,,,
2018-01-01 00:00:00,1.0,64.50,22.95,446.4,91.7125,36.4375,888.8
2018-01-01 01:00:00,1.0,64.35,23.25,446.4,91.1625,36.5750,888.8
2018-01-01 02:00:00,1.0,64.05,22.95,446.4,90.7500,36.4375,888.8
2018-01-01 03:00:00,1.0,63.60,22.80,446.4,93.5000,37.8125,888.8
2018-01-01 04:00:00,1.0,64.20,22.65,446.4,123.6125,51.7000,888.8
...,...,...,...,...,...,...,...
2020-12-31 19:00:00,1.0,53.55,26.40,NaN,137.3625,55.9625,NaN
2020-12-31 20:00:00,1.0,54.15,27.15,NaN,138.0500,56.9250,NaN
2020-12-31 21:00:00,1.0,54.30,26.70,NaN,136.8125,55.2750,NaN


In [16]:
df = fix_missing_values(df)

In [18]:
# df.to_csv('energia_cleaned_v2.csv')